# Training a classifier on GLCM features

In [1]:
import os
import re
import numpy as np
import nibabel as nib
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from pathlib import Path
import pandas as pd
from scipy.stats import ttest_ind
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, make_scorer
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, make_scorer
import scipy.stats as st

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Loading the data

Since we failed to implement the GLCM extractor in the previous Milestone we will load the data from the file uploaded to the website.

In [3]:
data = np.load("slice_glcm1d.npz", allow_pickle=True)

In [4]:
data.files

['slice_features', 'slice_meta', 'features_rankin_idx']

## Data exploration

In [5]:
df_features = pd.DataFrame(data['slice_features'])

In [6]:
df_features.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,3912.728918,1.096981e+07,38704.669443,2122.761852,236.130245,0.799733,8.958410,1.280547,153.858318,0.668218,0.655880,0.986222,0.941925,-0.382916,0.874570,0.000520,58.659495,0.149956,3.108626,0.818609,0.276677,117.318989,2.763986,589.723024
1,7511.568847,3.138924e+07,-19143.340098,4128.827862,188.601517,0.912418,8.283005,1.331034,117.985636,0.640403,0.624592,0.988914,0.944994,-0.483568,0.953910,0.000812,80.784854,0.085426,3.834779,0.918409,0.139321,161.569708,3.407020,1079.357345
2,8858.349761,3.493607e+07,-197109.877954,3355.497691,223.661489,0.874257,8.822815,1.381786,143.925152,0.639497,0.624148,0.987052,0.942295,-0.431027,0.919476,0.000775,89.861605,0.186931,3.461811,0.892283,0.405126,179.723210,3.044884,894.789795
3,9830.155057,1.601792e+07,-118166.648120,1794.384095,208.654995,0.789844,7.819523,1.297230,146.325976,0.689305,0.676263,0.988036,0.949242,-0.344585,0.826478,0.000654,97.126247,0.315186,2.823842,0.817015,0.548032,194.252495,2.435366,500.759772
4,9932.448352,2.210879e+07,-151369.529909,1948.802779,209.900402,0.803567,7.611905,1.277180,150.267237,0.705191,0.693021,0.987998,0.950864,-0.350157,0.824300,0.000604,97.455314,0.344902,2.748613,0.826901,0.577190,194.910628,2.368467,539.675795
5,8930.091000,3.680191e+07,-229102.274016,2713.806685,283.304445,0.809985,8.544302,1.295381,208.785424,0.721879,0.712215,0.984264,0.946661,-0.371537,0.833388,0.000393,91.226701,0.364181,2.653049,0.829366,0.594470,182.453403,2.349137,749.277782
6,8576.758367,2.732152e+07,-177708.643387,2787.006579,220.008199,0.851905,8.520492,1.367323,145.490819,0.656222,0.641698,0.987289,0.944433,-0.403192,0.889550,0.000730,89.075962,0.248289,3.158242,0.865426,0.479106,178.151925,2.751563,751.753694
7,6688.130517,2.877104e+07,-7656.743452,3900.444004,222.283235,0.892005,9.652227,1.419533,126.229764,0.583616,0.565366,0.986951,0.936002,-0.439675,0.942017,0.000936,75.949329,0.076954,3.953400,0.899283,0.139686,151.898658,3.485520,1030.681810
8,4092.571322,1.697277e+07,62260.408666,2630.453852,498.902036,0.682823,14.054398,1.308317,287.666878,0.596372,0.584484,0.972200,0.913752,-0.294606,0.792542,0.000391,59.649966,0.156543,3.079470,0.724717,0.258803,119.299931,2.584526,782.338972
9,2480.780137,2.004953e+07,82522.045776,2609.008164,347.669758,0.764538,12.623889,1.708628,175.898831,0.508736,0.488398,0.980172,0.918462,-0.310629,0.855313,0.001009,43.760201,0.099612,3.889895,0.767509,0.226552,87.520402,3.285765,739.169481


In [7]:
df_features.shape

(7414, 24)

In [8]:
for element in data['slice_meta']:
    print(element)

['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'NoNod']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_4' 3 4 'NoNod']
['LIDC-IDRI-0003_GT1_4' 3 4 'NoNod']
['L

In [9]:
df_meta = pd.DataFrame(data['slice_meta'], columns=['filename', 'patient_id', 'nodule_id', 'diagnosis'])


In [10]:
df_meta.head(100)

,filename,patient_id,nodule_id,diagnosis
0,LIDC-IDRI-0001_GT1_1,1,1,Malignant
1,LIDC-IDRI-0001_GT1_1,1,1,Malignant
2,LIDC-IDRI-0001_GT1_1,1,1,Malignant
3,LIDC-IDRI-0001_GT1_1,1,1,Malignant
4,LIDC-IDRI-0001_GT1_1,1,1,Malignant
5,LIDC-IDRI-0001_GT1_1,1,1,Malignant
6,LIDC-IDRI-0001_GT1_1,1,1,Malignant
7,LIDC-IDRI-0001_GT1_1,1,1,Malignant
8,LIDC-IDRI-0001_GT1_1,1,1,NoNod
9,LIDC-IDRI-0003_GT1_2,3,2,Malignant


In [11]:
df_meta["nodule_id"].unique()

array([1, 2, 3, 4, 6, 7, 9, 5, 10, 11, 12, 13, 14, 15, 8, 16, 17],
      dtype=object)

In [12]:
df_meta['diagnosis'].unique()

array(['Malignant', 'NoNod', 'Benign'], dtype=object)

In [13]:
labels = df_meta['diagnosis']
labels.head()

0    Malignant
1    Malignant
2    Malignant
3    Malignant
4    Malignant
Name: diagnosis, dtype: object

In [14]:
df_meta.shape

(7414, 4)

In [15]:
df_feature_ranking = pd.DataFrame(data['features_rankin_idx'])

In [16]:
df_feature_ranking.head(10)

,0
0,15
1,16
2,21
3,0
4,18
5,2
6,22
7,10
8,20
9,7


In [17]:
df_feature_ranking.shape

(24, 1)

## Getting rid of NoNods

In [18]:
df_binary = df_features[labels != 'NoNod']

In [19]:
df_binary.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,3912.728918,1.096981e+07,38704.669443,2122.761852,236.130245,0.799733,8.958410,1.280547,153.858318,0.668218,0.655880,0.986222,0.941925,-0.382916,0.874570,0.000520,58.659495,0.149956,3.108626,0.818609,0.276677,117.318989,2.763986,589.723024
1,7511.568847,3.138924e+07,-19143.340098,4128.827862,188.601517,0.912418,8.283005,1.331034,117.985636,0.640403,0.624592,0.988914,0.944994,-0.483568,0.953910,0.000812,80.784854,0.085426,3.834779,0.918409,0.139321,161.569708,3.407020,1079.357345
2,8858.349761,3.493607e+07,-197109.877954,3355.497691,223.661489,0.874257,8.822815,1.381786,143.925152,0.639497,0.624148,0.987052,0.942295,-0.431027,0.919476,0.000775,89.861605,0.186931,3.461811,0.892283,0.405126,179.723210,3.044884,894.789795
3,9830.155057,1.601792e+07,-118166.648120,1794.384095,208.654995,0.789844,7.819523,1.297230,146.325976,0.689305,0.676263,0.988036,0.949242,-0.344585,0.826478,0.000654,97.126247,0.315186,2.823842,0.817015,0.548032,194.252495,2.435366,500.759772
4,9932.448352,2.210879e+07,-151369.529909,1948.802779,209.900402,0.803567,7.611905,1.277180,150.267237,0.705191,0.693021,0.987998,0.950864,-0.350157,0.824300,0.000604,97.455314,0.344902,2.748613,0.826901,0.577190,194.910628,2.368467,539.675795


In [20]:
df_binary.shape

(2383, 24)

In [21]:
filt_nonod = df_meta[df_meta['diagnosis'] != 'NoNod']

In [22]:
filt_nonod.shape

(2383, 4)

In [23]:
labels = filt_nonod['diagnosis']

In [24]:
labels.unique()

array(['Malignant', 'Benign'], dtype=object)

## Performing the t-test for feature importance

Is this correct? What if we have feature correlations? 

In [25]:
# 'Malignant', 'NoNod', 'Benign'
class_0 = df_binary[labels == 'Benign']  # Subset where class is 0
class_1 = df_binary[labels == 'Malignant']  # Subset where class is 1

# Perform t-test for each feature
p_values = {col: ttest_ind(class_0[col], class_1[col], equal_var=True).pvalue for col in df_features.columns}

# Convert results to DataFrame
feature_importance = pd.DataFrame.from_dict(p_values, orient='index', columns=['p_value'])

# Sort by significance
feature_importance = feature_importance.sort_values(by='p_value')

print(feature_importance.head(24))  # Features with the smallest p-values

         p_value
15  1.041978e-35
16  6.250837e-08
21  6.250837e-08
0   5.315320e-07
18  1.193866e-05
2   1.818947e-05
22  2.126097e-05
10  2.684689e-04
20  3.112284e-04
7   3.398321e-04
9   7.353698e-04
8   8.400409e-03
17  1.363838e-02
5   5.373388e-02
11  5.518758e-02
4   5.645414e-02
23  7.537756e-02
6   8.396037e-02
12  1.022167e-01
3   1.220338e-01
19  2.335962e-01
1   6.855817e-01
14  7.524653e-01
13  8.305820e-01


#### Check if my t_tests are the same as the ones I loaded

In [26]:
all(df_feature_ranking[0] == feature_importance.index)

True

### Filter out p values > 0.05

In [27]:
important_features = feature_importance[feature_importance < 0.05]

In [28]:
important_features.shape

(24, 1)

In [29]:
important_features.head(24)

,p_value
15,1.041978e-35
16,6.250837e-08
21,6.250837e-08
0,5.315320e-07
18,1.193866e-05
2,1.818947e-05
22,2.126097e-05
10,2.684689e-04
20,3.112284e-04
7,3.398321e-04


In [30]:
important_features = important_features.dropna()
important_features.shape

(13, 1)

In [31]:
df_binary.columns

RangeIndex(start=0, stop=24, step=1)

In [32]:
df_binary_imp = df_binary[important_features.index]

In [33]:
df_binary_imp.columns

Index([15, 16, 21, 0, 18, 2, 22, 10, 20, 7, 9, 8, 17], dtype='int64')

In [34]:
len(df_binary_imp[labels=='Benign'])

245

In [35]:
len(df_binary_imp[labels=='Malignant'])

2138

In [36]:
df_binary_imp.head()

,15,16,21,0,18,2,22,10,20,7,9,8,17
0,0.000520,58.659495,117.318989,3912.728918,3.108626,38704.669443,2.763986,0.655880,0.276677,1.280547,0.668218,153.858318,0.149956
1,0.000812,80.784854,161.569708,7511.568847,3.834779,-19143.340098,3.407020,0.624592,0.139321,1.331034,0.640403,117.985636,0.085426
2,0.000775,89.861605,179.723210,8858.349761,3.461811,-197109.877954,3.044884,0.624148,0.405126,1.381786,0.639497,143.925152,0.186931
3,0.000654,97.126247,194.252495,9830.155057,2.823842,-118166.648120,2.435366,0.676263,0.548032,1.297230,0.689305,146.325976,0.315186
4,0.000604,97.455314,194.910628,9932.448352,2.748613,-151369.529909,2.368467,0.693021,0.577190,1.277180,0.705191,150.267237,0.344902


## Training SVM on Stratified k-fold

In [51]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model_base = SVC(class_weight='balanced')
scoring = {
    'accuracy': 'accuracy',
    'precision_malignant': make_scorer(precision_score, pos_label='Malignant'),
    'recall_malignant': make_scorer(recall_score, pos_label='Malignant'),
    'f1_malignant': make_scorer(f1_score, pos_label='Malignant'),
    'precision_benign': make_scorer(precision_score, pos_label='Benign'),
    'recall_benign': make_scorer(recall_score, pos_label='Benign'),
    'f1_benign': make_scorer(f1_score, pos_label='Benign')
}

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_binary_imp)

# Run cross-validation
results = cross_validate(
    model_base,
    X_scaled,
    labels,
    cv=cv,
    scoring=scoring,
    n_jobs=-1
)

# Print full results
print("\n=== Detailed Metrics Per Fold ===")
for fold in range(10):
    print(f"\nFold {fold + 1}:")
    print(f"  Accuracy: {results['test_accuracy'][fold]:.4f}")
    print("  Malignant:")
    print(f"    Precision: {results['test_precision_malignant'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_malignant'][fold]:.4f}")
    print(f"    F1: {results['test_f1_malignant'][fold]:.4f}")
    print("  Benign:")
    print(f"    Precision: {results['test_precision_benign'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_benign'][fold]:.4f}")
    print(f"    F1: {results['test_f1_benign'][fold]:.4f}")

# Print averages
print("\n=== Average Metrics ===")
print(f"Accuracy: {np.mean(results['test_accuracy']):.4f} (±{np.std(results['test_accuracy']):.4f})")
print("\nMalignant:")
print(f"  Precision: {np.mean(results['test_precision_malignant']):.4f} (±{np.std(results['test_precision_malignant']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_malignant']):.4f} (±{np.std(results['test_recall_malignant']):.4f})")
print(f"  F1: {np.mean(results['test_f1_malignant']):.4f} (±{np.std(results['test_f1_malignant']):.4f})")
print("\nBenign:")
print(f"  Precision: {np.mean(results['test_precision_benign']):.4f} (±{np.std(results['test_precision_benign']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_benign']):.4f} (±{np.std(results['test_recall_benign']):.4f})")
print(f"  F1: {np.mean(results['test_f1_benign']):.4f} (±{np.std(results['test_f1_benign']):.4f})")


=== Detailed Metrics Per Fold ===

Fold 1:
  Accuracy: 0.8201
  Malignant:
    Precision: 0.9672
    Recall: 0.8271
    F1: 0.8917
  Benign:
    Precision: 0.3393
    Recall: 0.7600
    F1: 0.4691

Fold 2:
  Accuracy: 0.8452
  Malignant:
    Precision: 0.9634
    Recall: 0.8598
    F1: 0.9086
  Benign:
    Precision: 0.3750
    Recall: 0.7200
    F1: 0.4932

Fold 3:
  Accuracy: 0.8285
  Malignant:
    Precision: 0.9436
    Recall: 0.8598
    F1: 0.8998
  Benign:
    Precision: 0.3182
    Recall: 0.5600
    F1: 0.4058

Fold 4:
  Accuracy: 0.8697
  Malignant:
    Precision: 0.9741
    Recall: 0.8785
    F1: 0.9238
  Benign:
    Precision: 0.4222
    Recall: 0.7917
    F1: 0.5507

Fold 5:
  Accuracy: 0.8655
  Malignant:
    Precision: 0.9505
    Recall: 0.8972
    F1: 0.9231
  Benign:
    Precision: 0.3889
    Recall: 0.5833
    F1: 0.4667

Fold 6:
  Accuracy: 0.8235
  Malignant:
    Precision: 0.9479
    Recall: 0.8505
    F1: 0.8966
  Benign:
    Precision: 0.3043
    Recall: 0.5833
  

In [52]:
# Extract cross-validation scores
cv_scores = results['test_accuracy']

# Compute mean and standard error
mean_score = np.mean(cv_scores)
std_error = st.sem(cv_scores)  # Standard Error of the Mean (SEM)

# Calculate confidence interval
confidence = 0.95
ci_lower, ci_upper = st.t.interval(confidence, len(cv_scores)-1, loc=mean_score, scale=std_error)

print(f"Mean Score: {mean_score:.4f}")
print(f"95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")

Mean Score: 0.8410
95% Confidence Interval: (0.8270, 0.8549)


In my opinion the model performs well, the most important factor from a patient's perspective, ,Did I find the tumors that are actually Malignant"? (Recall) is almost 100%. However, we can improve on the other metrics to reduce the chance of unnecessary bad news for the patients and eliminate / reduce the cost of further diagnoses.

The accuracy is also not bad, around 90% with pretty high confidence

In [53]:
filt_nonod.head()

,filename,patient_id,nodule_id,diagnosis
0,LIDC-IDRI-0001_GT1_1,1,1,Malignant
1,LIDC-IDRI-0001_GT1_1,1,1,Malignant
2,LIDC-IDRI-0001_GT1_1,1,1,Malignant
3,LIDC-IDRI-0001_GT1_1,1,1,Malignant
4,LIDC-IDRI-0001_GT1_1,1,1,Malignant


In [54]:
filt_nonod.shape

(2383, 4)

In [55]:
nodules = filt_nonod['nodule_id']

In [56]:
nodules.unique()

array([1, 2, 4, 3, 6, 5, 10, 8, 7, 11, 12], dtype=object)

### NOT GOOD, WE HAVE TO DO THE GROUPING BY THE FILE NAME

In [57]:
nodules = filt_nonod["filename"]

In [58]:
len(nodules)

2383

In [59]:
len(labels)

2383

## Training SVM on Stratified Group K-Fold

In [61]:
cv = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=42)
model_base = SVC(class_weight='balanced')
scoring = {
    'accuracy': 'accuracy',
    'precision_malignant': make_scorer(precision_score, pos_label='Malignant'),
    'recall_malignant': make_scorer(recall_score, pos_label='Malignant'),
    'f1_malignant': make_scorer(f1_score, pos_label='Malignant'),
    'precision_benign': make_scorer(precision_score, pos_label='Benign'),
    'recall_benign': make_scorer(recall_score, pos_label='Benign'),
    'f1_benign': make_scorer(f1_score, pos_label='Benign')
}

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_binary_imp)

# Run cross-validation
results = cross_validate(
    model_base,
    X_scaled,
    labels,
    cv=cv,
    scoring=scoring,
    groups=nodules,
    n_jobs=-1
)

# Print full results
print("\n=== Detailed Metrics Per Fold ===")
for fold in range(10):
    print(f"\nFold {fold + 1}:")
    print(f"  Accuracy: {results['test_accuracy'][fold]:.4f}")
    print("  Malignant:")
    print(f"    Precision: {results['test_precision_malignant'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_malignant'][fold]:.4f}")
    print(f"    F1: {results['test_f1_malignant'][fold]:.4f}")
    print("  Benign:")
    print(f"    Precision: {results['test_precision_benign'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_benign'][fold]:.4f}")
    print(f"    F1: {results['test_f1_benign'][fold]:.4f}")

# Print averages
print("\n=== Average Metrics ===")
print(f"Accuracy: {np.mean(results['test_accuracy']):.4f} (±{np.std(results['test_accuracy']):.4f})")
print("\nMalignant:")
print(f"  Precision: {np.mean(results['test_precision_malignant']):.4f} (±{np.std(results['test_precision_malignant']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_malignant']):.4f} (±{np.std(results['test_recall_malignant']):.4f})")
print(f"  F1: {np.mean(results['test_f1_malignant']):.4f} (±{np.std(results['test_f1_malignant']):.4f})")
print("\nBenign:")
print(f"  Precision: {np.mean(results['test_precision_benign']):.4f} (±{np.std(results['test_precision_benign']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_benign']):.4f} (±{np.std(results['test_recall_benign']):.4f})")
print(f"  F1: {np.mean(results['test_f1_benign']):.4f} (±{np.std(results['test_f1_benign']):.4f})")


=== Detailed Metrics Per Fold ===

Fold 1:
  Accuracy: 0.6894
  Malignant:
    Precision: 0.8916
    Recall: 0.7291
    F1: 0.8022
  Benign:
    Precision: 0.2029
    Recall: 0.4375
    F1: 0.2772

Fold 2:
  Accuracy: 0.8582
  Malignant:
    Precision: 0.9042
    Recall: 0.9440
    F1: 0.9237
  Benign:
    Precision: 0.0000
    Recall: 0.0000
    F1: 0.0000

Fold 3:
  Accuracy: 0.7245
  Malignant:
    Precision: 0.9141
    Recall: 0.7637
    F1: 0.8322
  Benign:
    Precision: 0.1642
    Recall: 0.3929
    F1: 0.2316

Fold 4:
  Accuracy: 0.9333
  Malignant:
    Precision: 1.0000
    Recall: 0.9268
    F1: 0.9620
  Benign:
    Precision: 0.5714
    Recall: 1.0000
    F1: 0.7273

Fold 5:
  Accuracy: 0.8163
  Malignant:
    Precision: 0.9222
    Recall: 0.8426
    F1: 0.8806
  Benign:
    Precision: 0.5231
    Recall: 0.7083
    F1: 0.6018

Fold 6:
  Accuracy: 0.7739
  Malignant:
    Precision: 0.9613
    Recall: 0.7945
    F1: 0.8700
  Benign:
    Precision: 0.0816
    Recall: 0.3636
  

In [62]:
# Extract cross-validation scores
cv_scores = results['test_accuracy']

# Compute mean and standard error
mean_score = np.mean(cv_scores)
std_error = st.sem(cv_scores)  # Standard Error of the Mean (SEM)

# Calculate confidence interval
confidence = 0.95
ci_lower, ci_upper = st.t.interval(confidence, len(cv_scores)-1, loc=mean_score, scale=std_error)

print(f"Mean Score: {mean_score:.4f}")
print(f"95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")

Mean Score: 0.8091
95% Confidence Interval: (0.7595, 0.8587)


In [48]:
# cv = StratifiedGroupKFold(n_splits=5)

# for fold, (train_idx, test_idx) in enumerate(cv.split(df_binary_imp, labels, nodules)):
#     print(f"\n--- Fold {fold + 1} ---")
#     X_train, X_test = df_binary_imp.iloc[train_idx], df_binary_imp.iloc[test_idx]
#     y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

#     # Feature Scaling
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)

#     clf = SVC()
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     # report = classification_report(y_test, y_pred, digits=4, output_dict=True)
#     # metrics.append(report)
#     # 
#     report = classification_report(y_test, y_pred, output_dict=True)
#     print(report)

#     # Flatten the dictionary into a DataFrame row
#     report_df = pd.DataFrame(report).transpose()
#     # report_df['Fold'] = fold  # Add fold number

#     metrics.append(report_df)

# # Combine all fold reports into one DataFrame
# df_metrics = pd.concat(metrics, axis=0).reset_index()

# # Compute average metrics across folds (excluding support count)
# avg_metrics = df_metrics.groupby('index').mean(numeric_only=True)

# print("____________________________________________")
# print("average metrics: ")
# print(avg_metrics)  # Displays properly formatted averaged classification report

### Conclusion:

This method is useful when you have grouped data, like medical records where multiple samples belong to the same patient. It prevents data leakage by ensuring that all samples from a single group are either in the training or test set, but never both.

Our results are slightly worse, possibly because of the small ammout of data leakage prevented

## Hyperparameter tuning

1. Purpose of cv_outer (Outer Loop)
What it does: Splits the data into training + validation (inner loop) and test (holdout) sets.

Why it's needed:

To evaluate the generalization performance of the model trained with the best hyperparameters.

Each fold in the outer loop gives an independent test set to avoid optimistic bias in performance metrics.

2. Purpose of cv_inner (Inner Loop)
What it does: Further splits the training set (from cv_outer) into smaller training/validation sets for hyperparameter tuning (e.g., Grid Search/Random Search/Optuna).

Why it's needed:

To select the best hyperparameters without peeking at the outer test set (prevents data leakage).

Ensures the model is tuned fairly within each outer fold.

### Grid search

In [49]:
xxx to stop from recalculating the upcoming grid search

SyntaxError: invalid syntax (1505234791.py, line 1)

In [68]:
cv_outer = StratifiedGroupKFold(n_splits=10)
cv_inner = StratifiedGroupKFold(n_splits=10)

In [69]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_binary_imp)

In [65]:
# Define parameter grid
param_grid = {
    'svc__C': [0.01, 0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto']
}

metrics = []

pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC(class_weight='balanced'))])

# Initialize grid search with cv_inner
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv_inner,  # Inner CV for hyperparameter tuning
    scoring='accuracy',
    n_jobs=-1,
    refit=True, # Refit the best model on the full dataset
    verbose = 1  # get some intermediate info about the training
)

# Fit on ALL data (handles train/test splits internally)
grid_search.fit(df_binary_imp, labels, groups=nodules)

# Best model and params
print("Best params:", grid_search.best_params_)
best_model = grid_search.best_estimator_



Fitting 10 folds for each of 20 candidates, totalling 200 fits


Best params: {'svc__C': 0.01, 'svc__gamma': 'scale', 'svc__kernel': 'linear'}


In [72]:
best_model = SVC(C=0.01, gamma='scale',kernel='linear', class_weight='balanced')

In [73]:

scoring = {
    'accuracy': 'accuracy',
    'precision_malignant': make_scorer(precision_score, pos_label='Malignant'),
    'recall_malignant': make_scorer(recall_score, pos_label='Malignant'),
    'f1_malignant': make_scorer(f1_score, pos_label='Malignant'),
    'precision_benign': make_scorer(precision_score, pos_label='Benign'),
    'recall_benign': make_scorer(recall_score, pos_label='Benign'),
    'f1_benign': make_scorer(f1_score, pos_label='Benign')
}

# Run cross-validation
results = cross_validate(
    best_model,
    X_scaled,
    labels,
    cv=cv_outer,
    groups=nodules,
    scoring=scoring,
    n_jobs=-1
)

# Print full results
print("\n=== Detailed Metrics Per Fold ===")
for fold in range(10):
    print(f"\nFold {fold + 1}:")
    print(f"  Accuracy: {results['test_accuracy'][fold]:.4f}")
    print("  Malignant:")
    print(f"    Precision: {results['test_precision_malignant'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_malignant'][fold]:.4f}")
    print(f"    F1: {results['test_f1_malignant'][fold]:.4f}")
    print("  Benign:")
    print(f"    Precision: {results['test_precision_benign'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_benign'][fold]:.4f}")
    print(f"    F1: {results['test_f1_benign'][fold]:.4f}")

# Print averages
print("\n=== Average Metrics ===")
print(f"Accuracy: {np.mean(results['test_accuracy']):.4f} (±{np.std(results['test_accuracy']):.4f})")
print("\nMalignant:")
print(f"  Precision: {np.mean(results['test_precision_malignant']):.4f} (±{np.std(results['test_precision_malignant']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_malignant']):.4f} (±{np.std(results['test_recall_malignant']):.4f})")
print(f"  F1: {np.mean(results['test_f1_malignant']):.4f} (±{np.std(results['test_f1_malignant']):.4f})")
print("\nBenign:")
print(f"  Precision: {np.mean(results['test_precision_benign']):.4f} (±{np.std(results['test_precision_benign']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_benign']):.4f} (±{np.std(results['test_recall_benign']):.4f})")
print(f"  F1: {np.mean(results['test_f1_benign']):.4f} (±{np.std(results['test_f1_benign']):.4f})")


=== Detailed Metrics Per Fold ===

Fold 1:
  Accuracy: 0.8912
  Malignant:
    Precision: 0.9563
    Recall: 0.9206
    F1: 0.9381
  Benign:
    Precision: 0.4848
    Recall: 0.6400
    F1: 0.5517

Fold 2:
  Accuracy: 0.8577
  Malignant:
    Precision: 0.9091
    Recall: 0.9346
    F1: 0.9217
  Benign:
    Precision: 0.2632
    Recall: 0.2000
    F1: 0.2273

Fold 3:
  Accuracy: 0.8912
  Malignant:
    Precision: 0.9273
    Recall: 0.9533
    F1: 0.9401
  Benign:
    Precision: 0.4737
    Recall: 0.3600
    F1: 0.4091

Fold 4:
  Accuracy: 0.8908
  Malignant:
    Precision: 0.9087
    Recall: 0.9766
    F1: 0.9414
  Benign:
    Precision: 0.3750
    Recall: 0.1250
    F1: 0.1875

Fold 5:
  Accuracy: 0.8571
  Malignant:
    Precision: 0.9202
    Recall: 0.9202
    F1: 0.9202
  Benign:
    Precision: 0.3200
    Recall: 0.3200
    F1: 0.3200

Fold 6:
  Accuracy: 0.8109
  Malignant:
    Precision: 0.8962
    Recall: 0.8920
    F1: 0.8941
  Benign:
    Precision: 0.1154
    Recall: 0.1200
  

In [74]:
# Extract cross-validation scores
cv_scores = results['test_accuracy']

# Compute mean and standard error
mean_score = np.mean(cv_scores)
std_error = st.sem(cv_scores)  # Standard Error of the Mean (SEM)

# Calculate confidence interval
confidence = 0.95
ci_lower, ci_upper = st.t.interval(confidence, len(cv_scores)-1, loc=mean_score, scale=std_error)

print(f"Mean Score: {mean_score:.4f}")
print(f"95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")

Mean Score: 0.8657
95% Confidence Interval: (0.8437, 0.8877)


### Random search

In [75]:
# Define parameter distributions
param_dist = {
    'svc__C': loguniform(1e-3, 1e3),
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 5)),
    'svc__shrinking': [True, False],  # Use shrinking heuristic (default=True)
    'svc__probability': [True, False],  # Enable probability estimates (default=False)
    'svc__tol': [1e-3, 1e-4, 1e-5],  # Tolerance for stopping (default=1e-3)
    'svc__class_weight': [None, 'balanced']  # Handle imbalanced classes (default=None)
}

metrics = []

pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC(class_weight='balanced'))])

# Initialize RandomizedSearchCV with OUTER CV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=20,  # Number of random combinations to try
    cv=cv_inner,  # Inner CV for hyperparameter tuning
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=2,
    refit=True  # Refit the best model on the full dataset
)

# Fit on ALL data (handles train/test splits internally)
random_search.fit(X_scaled, labels, groups=nodules)

# Best model and params
print("Best params:", random_search.best_params_)
best_model_r = random_search.best_estimator_

Fitting 10 folds for each of 20 candidates, totalling 200 fits
Best params: {'svc__C': np.float64(99.80894623678049), 'svc__class_weight': None, 'svc__gamma': np.float64(31.622776601683793), 'svc__kernel': 'linear', 'svc__probability': False, 'svc__shrinking': False, 'svc__tol': 0.0001}


Save the model for not having to retrain in case of crash

In [76]:
from joblib import dump
from joblib import load

# Save the model to a file
dump(best_model_r, 'best_model_05.23_10fold_BALANCED.joblib')

# Later, load it back

bm = load('best_model_05.23_10fold_BALANCED.joblib')

In [ ]:
from joblib import load
best_model_r = load('best_model_05.2310fold_BALANCED.joblib')

In [77]:
# Initialize RandomizedSearchCV with OUTER CV. We could not go with the previous way as the grid search
# because we want to avoid changing the model randomly between each fold


scoring = {
    'accuracy': 'accuracy',
    'precision_malignant': make_scorer(precision_score, pos_label='Malignant'),
    'recall_malignant': make_scorer(recall_score, pos_label='Malignant'),
    'f1_malignant': make_scorer(f1_score, pos_label='Malignant'),
    'precision_benign': make_scorer(precision_score, pos_label='Benign'),
    'recall_benign': make_scorer(recall_score, pos_label='Benign'),
    'f1_benign': make_scorer(f1_score, pos_label='Benign')
}

# Run cross-validation
results = cross_validate(
    best_model_r,
    X_scaled,
    labels,
    cv=cv_outer,
    groups=nodules,
    scoring=scoring,
    n_jobs=-1
)

# Print full results
print("\n=== Detailed Metrics Per Fold ===")
for fold in range(10):
    print(f"\nFold {fold + 1}:")
    print(f"  Accuracy: {results['test_accuracy'][fold]:.4f}")
    print("  Malignant:")
    print(f"    Precision: {results['test_precision_malignant'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_malignant'][fold]:.4f}")
    print(f"    F1: {results['test_f1_malignant'][fold]:.4f}")
    print("  Benign:")
    print(f"    Precision: {results['test_precision_benign'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_benign'][fold]:.4f}")
    print(f"    F1: {results['test_f1_benign'][fold]:.4f}")

# Print averages
print("\n=== Average Metrics ===")
print(f"Accuracy: {np.mean(results['test_accuracy']):.4f} (±{np.std(results['test_accuracy']):.4f})")
print("\nMalignant:")
print(f"  Precision: {np.mean(results['test_precision_malignant']):.4f} (±{np.std(results['test_precision_malignant']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_malignant']):.4f} (±{np.std(results['test_recall_malignant']):.4f})")
print(f"  F1: {np.mean(results['test_f1_malignant']):.4f} (±{np.std(results['test_f1_malignant']):.4f})")
print("\nBenign:")
print(f"  Precision: {np.mean(results['test_precision_benign']):.4f} (±{np.std(results['test_precision_benign']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_benign']):.4f} (±{np.std(results['test_recall_benign']):.4f})")
print(f"  F1: {np.mean(results['test_f1_benign']):.4f} (±{np.std(results['test_f1_benign']):.4f})")


=== Detailed Metrics Per Fold ===

Fold 1:
  Accuracy: 0.9289
  Malignant:
    Precision: 0.9264
    Recall: 1.0000
    F1: 0.9618
  Benign:
    Precision: 1.0000
    Recall: 0.3200
    F1: 0.4848

Fold 2:
  Accuracy: 0.9163
  Malignant:
    Precision: 0.9145
    Recall: 1.0000
    F1: 0.9554
  Benign:
    Precision: 1.0000
    Recall: 0.2000
    F1: 0.3333

Fold 3:
  Accuracy: 0.8912
  Malignant:
    Precision: 0.8950
    Recall: 0.9953
    F1: 0.9425
  Benign:
    Precision: 0.0000
    Recall: 0.0000
    F1: 0.0000

Fold 4:
  Accuracy: 0.8992
  Malignant:
    Precision: 0.8992
    Recall: 1.0000
    F1: 0.9469
  Benign:
    Precision: 0.0000
    Recall: 0.0000
    F1: 0.0000

Fold 5:
  Accuracy: 0.9034
  Malignant:
    Precision: 0.9025
    Recall: 1.0000
    F1: 0.9488
  Benign:
    Precision: 1.0000
    Recall: 0.0800
    F1: 0.1481

Fold 6:
  Accuracy: 0.8361
  Malignant:
    Precision: 0.8884
    Recall: 0.9343
    F1: 0.9108
  Benign:
    Precision: 0.0000
    Recall: 0.0000
  

In [78]:
# Extract cross-validation scores
cv_scores = results['test_accuracy']

# Compute mean and standard error
mean_score = np.mean(cv_scores)
std_error = st.sem(cv_scores)  # Standard Error of the Mean (SEM)

# Calculate confidence interval
confidence = 0.95
ci_lower, ci_upper = st.t.interval(confidence, len(cv_scores)-1, loc=mean_score, scale=std_error)

print(f"Mean Score: {mean_score:.4f}")
print(f"95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")

Mean Score: 0.9039
95% Confidence Interval: (0.8830, 0.9248)


The accuracies are again a little bit worse...

### Optuna

In [85]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import accuracy_score

def objective(trial, X_train, y_train, groups_train):
    # Hyperparameters to optimize
    C = trial.suggest_float("C", 1e-3, 1e3, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf"])
    
    # Conditional parameters
    params = {
        "C": C,
        "kernel": kernel,
        "shrinking": trial.suggest_categorical("shrinking", [True, False]),
        "tol": trial.suggest_float("tol", 1e-5, 1e-1, log=True)
    }

    # Model training and validation
    model = SVC(**params, random_state=42, class_weight='balanced')
    scores = cross_val_score(
        model, X_train, y_train, cv=cv_inner, groups=groups_train, scoring="accuracy"
    )
    return scores.mean()


# Optuna study
study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
study.optimize(lambda trial: objective(trial, X_scaled, labels, nodules), n_trials=200)

# Get best model
best_params = study.best_params
best_model = SVC(**best_params, random_state=42,class_weight='balanced')


[I 2025-05-23 17:46:19,548] A new study created in memory with name: no-name-1d4d525b-bd20-443d-88c3-dad44c7328f9
[I 2025-05-23 17:46:21,207] Trial 0 finished with value: 0.8254140149783764 and parameters: {'C': 0.1767016940294795, 'kernel': 'linear', 'shrinking': True, 'tol': 4.207053950287933e-05}. Best is trial 0 with value: 0.8254140149783764.
[I 2025-05-23 17:46:22,908] Trial 1 finished with value: 0.8199641362821278 and parameters: {'C': 0.002231010801867922, 'kernel': 'linear', 'shrinking': True, 'tol': 0.07579479953348005}. Best is trial 0 with value: 0.8254140149783764.
[I 2025-05-23 17:49:41,177] Trial 2 finished with value: 0.8014872894764601 and parameters: {'C': 98.77700294007911, 'kernel': 'linear', 'shrinking': False, 'tol': 0.0012561043700013551}. Best is trial 0 with value: 0.8254140149783764.
[I 2025-05-23 17:49:43,362] Trial 3 finished with value: 0.8204001265778278 and parameters: {'C': 0.3905441275210791, 'kernel': 'rbf', 'shrinking': False, 'tol': 0.00029204338471

In [86]:
print(best_model)

SVC(C=0.004820857053872796, class_weight='balanced', random_state=42,
    tol=0.023873473888613835)


(I could use more trials, but for some reason this tool crashes after 9-14. Keeps running but nothing happens. SO I stay with 10)

In [87]:
# Initialize RandomizedSearchCV with OUTER CV. We could not go with the previous way as the grid search
# because we want to avoid changing the model randomly between each fold


scoring = {
    'accuracy': 'accuracy',
    'precision_malignant': make_scorer(precision_score, pos_label='Malignant'),
    'recall_malignant': make_scorer(recall_score, pos_label='Malignant'),
    'f1_malignant': make_scorer(f1_score, pos_label='Malignant'),
    'precision_benign': make_scorer(precision_score, pos_label='Benign'),
    'recall_benign': make_scorer(recall_score, pos_label='Benign'),
    'f1_benign': make_scorer(f1_score, pos_label='Benign')
}

# Run cross-validation
results = cross_validate(
    best_model,
    X_scaled,
    labels,
    cv=cv_outer,
    groups=nodules,
    scoring=scoring,
    n_jobs=-1
)

# Print full results
print("\n=== Detailed Metrics Per Fold ===")
for fold in range(5):
    print(f"\nFold {fold + 1}:")
    print(f"  Accuracy: {results['test_accuracy'][fold]:.4f}")
    print("  Malignant:")
    print(f"    Precision: {results['test_precision_malignant'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_malignant'][fold]:.4f}")
    print(f"    F1: {results['test_f1_malignant'][fold]:.4f}")
    print("  Benign:")
    print(f"    Precision: {results['test_precision_benign'][fold]:.4f}")
    print(f"    Recall: {results['test_recall_benign'][fold]:.4f}")
    print(f"    F1: {results['test_f1_benign'][fold]:.4f}")

# Print averages
print("\n=== Average Metrics ===")
print(f"Accuracy: {np.mean(results['test_accuracy']):.4f} (±{np.std(results['test_accuracy']):.4f})")
print("\nMalignant:")
print(f"  Precision: {np.mean(results['test_precision_malignant']):.4f} (±{np.std(results['test_precision_malignant']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_malignant']):.4f} (±{np.std(results['test_recall_malignant']):.4f})")
print(f"  F1: {np.mean(results['test_f1_malignant']):.4f} (±{np.std(results['test_f1_malignant']):.4f})")
print("\nBenign:")
print(f"  Precision: {np.mean(results['test_precision_benign']):.4f} (±{np.std(results['test_precision_benign']):.4f})")
print(f"  Recall: {np.mean(results['test_recall_benign']):.4f} (±{np.std(results['test_recall_benign']):.4f})")
print(f"  F1: {np.mean(results['test_f1_benign']):.4f} (±{np.std(results['test_f1_benign']):.4f})")


=== Detailed Metrics Per Fold ===

Fold 1:
  Accuracy: 0.9331
  Malignant:
    Precision: 0.9304
    Recall: 1.0000
    F1: 0.9640
  Benign:
    Precision: 1.0000
    Recall: 0.3600
    F1: 0.5294

Fold 2:
  Accuracy: 0.9079
  Malignant:
    Precision: 0.9138
    Recall: 0.9907
    F1: 0.9507
  Benign:
    Precision: 0.7143
    Recall: 0.2000
    F1: 0.3125

Fold 3:
  Accuracy: 0.8828
  Malignant:
    Precision: 0.8941
    Recall: 0.9860
    F1: 0.9378
  Benign:
    Precision: 0.0000
    Recall: 0.0000
    F1: 0.0000

Fold 4:
  Accuracy: 0.8992
  Malignant:
    Precision: 0.8992
    Recall: 1.0000
    F1: 0.9469
  Benign:
    Precision: 0.0000
    Recall: 0.0000
    F1: 0.0000

Fold 5:
  Accuracy: 0.9118
  Malignant:
    Precision: 0.9103
    Recall: 1.0000
    F1: 0.9530
  Benign:
    Precision: 1.0000
    Recall: 0.1600
    F1: 0.2759

=== Average Metrics ===
Accuracy: 0.9026 (±0.0317)

Malignant:
  Precision: 0.9105 (±0.0154)
  Recall: 0.9887 (±0.0249)
  F1: 0.9479 (±0.0176)

Benig

In [88]:
# Extract cross-validation scores
cv_scores = results['test_accuracy']

# Compute mean and standard error
mean_score = np.mean(cv_scores)
std_error = st.sem(cv_scores)  # Standard Error of the Mean (SEM)

# Calculate confidence interval
confidence = 0.95
ci_lower, ci_upper = st.t.interval(confidence, len(cv_scores)-1, loc=mean_score, scale=std_error)

print(f"Mean Score: {mean_score:.4f}")
print(f"95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")

Mean Score: 0.9026
95% Confidence Interval: (0.8787, 0.9266)


THe metrics are quite similar to the random search. 